#P0. Introducción -PLN

---

En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

*   Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
*   Las de memoria corta (Long Short Term Memory)...PLN
*   Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.



**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.


---
https://unipython.com/generacion-de-textos-con-inteligencia-artificial/


#EJEMPLO 1: Prediciendo un texto (cuento los tres cerditos).

##P0. importando librerias

###Librerias genericas

In [ ]:
!pip list

In [1]:
import requests
import io
import os

import numpy as np
import sys

#librerías para graficar
import matplotlib.pyplot as plt
                 
plt.rcParams['figure.figsize'] = (16, 9)  #ver graficas grandes 
plt.style.use('ggplot')
#guardar las imagenes y tablas en el cuaderno de jupyter
%matplotlib inline 

###librerias para DL (Deep Learning)

In [2]:
import tensorflow as tf

In [3]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
GPU is available


###Activando la GPU

---
Logra aumentar la velocidad de entrenamiento en un 600% en PLN (RNN) y un 1000% en visión por computadores (CNN)


In [4]:
tf.device('/cpu:0')# OJO: esto es para que tensorflow utilice la GPU (aumenta la velocidad de entrenamiento en un 600%) 

##P1. Descarga y preprocesado de los datos

In [5]:
fileUrl     ='https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/Datasets/Panel_Txt_Files/Tres_cerditos.txt'
fileContent = tf.keras.utils.get_file('Tres_cerditos.txt',fileUrl)
texto       = open(fileContent, 'rb').read().decode(encoding='utf-8')
raw_text    = texto.lower()
print(raw_text)

16384/3231 [========================================================================================================================================================] - 0s 0us/step
había una vez tres cerditos que eran hermanos y se fueron por el mundo a conseguir fortuna. el más grande les dijo a sus hermanos que sería bueno que se pusieran a construir sus propias casas para estar protegidos. a los otros dos les pareció una buena idea, y se pusieron manos a la obra, cada uno construyó su casita.  - la mía será de paja - dijo el más pequeño-, la paja es blanda y se puede sujetar con facilidad. terminaré muy pronto y podré ir a jugar. el hermano mediano decidió que su casa sería de madera:  - puedo encontrar un montón de madera por los alrededores - explicó 
a sus hermanos, - construiré mi casa en un santiamén con todos estos troncos y me iré también a jugar.  cuando las tres casitas estuvieron terminadas, los cerditos cantaban y bailaban en la puerta, felices por haber acabado con el pro

##P2. pasar el texto a números

---
Sin importan el origen de la informacción (video, sonido, sensor, texto)...siempre debemos convertirlos en datos númericos.


In [6]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
print(char_to_int)

['\n', '\r', ' ', '!', ',', '-', '.', ':', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', 'á', 'é', 'í', 'ñ', 'ó', 'ú']
{'\n': 0, '\r': 1, ' ': 2, '!': 3, ',': 4, '-': 5, '.': 6, ':': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'f': 13, 'g': 14, 'h': 15, 'i': 16, 'j': 17, 'l': 18, 'm': 19, 'n': 20, 'o': 21, 'p': 22, 'q': 23, 'r': 24, 's': 25, 't': 26, 'u': 27, 'v': 28, 'x': 29, 'y': 30, 'z': 31, '¡': 32, 'á': 33, 'é': 34, 'í': 35, 'ñ': 36, 'ó': 37, 'ú': 38}


In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))

En total hay 3141 caracteres, y el diccionario tiene un tamaño de 39 caracteres.


###P2.1 Dividimos el texto en secuencias:
---
Dividimos el texto en estas secuencias (adrede), convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente



In [8]:
# preparar el conjunto de datos de los pares de entrada a salida codificados como enteros
seq_length = 50   #largo de las secciones de texto que usaremos para entrenar
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	print("Seq(",i,")=",raw_text[i:i + seq_length],"---->",raw_text[i + seq_length])
	seq_in 	= raw_text[i:i + seq_length]         					#secuencia de entrada
	seq_out = raw_text[i + seq_length]										#siguiente letra despues de la secuencia (la que el va a aprender)
	dataX.append([char_to_int[char] for char in seq_in])  # convertimos cada secuencia en numeros
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Se generaron ",format(n_patterns) ," secuencias texto de un tamaño de ",seq_length," caracteres" )
print("Como se ven los datos de X convertidos a números\n")
print(dataX)
print("\nComo se ven los datos de Y convertidos a números\n")
print(dataY)

Seq( 0 )= había una vez tres cerditos que eran hermanos y se ---->  
Seq( 1 )= abía una vez tres cerditos que eran hermanos y se  ----> f
Seq( 2 )= bía una vez tres cerditos que eran hermanos y se f ----> u
Seq( 3 )= ía una vez tres cerditos que eran hermanos y se fu ----> e
Seq( 4 )= a una vez tres cerditos que eran hermanos y se fue ----> r
Seq( 5 )=  una vez tres cerditos que eran hermanos y se fuer ----> o
Seq( 6 )= una vez tres cerditos que eran hermanos y se fuero ----> n
Seq( 7 )= na vez tres cerditos que eran hermanos y se fueron ---->  
Seq( 8 )= a vez tres cerditos que eran hermanos y se fueron  ----> p
Seq( 9 )=  vez tres cerditos que eran hermanos y se fueron p ----> o
Seq( 10 )= vez tres cerditos que eran hermanos y se fueron po ----> r
Seq( 11 )= ez tres cerditos que eran hermanos y se fueron por ---->  
Seq( 12 )= z tres cerditos que eran hermanos y se fueron por  ----> e
Seq( 13 )=  tres cerditos que eran hermanos y se fueron por e ----> l
Seq( 14 )= tres cerditos que e

##P3. preparar nuestros datos de entrenamiento

---


1.   Primero debemos transformar la lista de secuencias de entrada en la forma [muestras, pasos de tiempo, características] esperada por una red LSTM.
2.   Luego debemos cambiar la escala de los números enteros al rango de 0 a 1 para que los patrones sean más fáciles de aprender mediante la red LSTM que utiliza la función de activación sigmoidea de forma predeterminada.
3.   por ultimo necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 54 caracteres diferentes en el vocabulario (una representación más fácil)







In [10]:
#transformar la lista X de secuencias de entrada en la forma [muestras (3091), pasos de tiempo, características]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalizar (cambiar la escala de los números enteros al rango de 0 a 1 )
X = X / float(n_vocab)
# convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot.
y = tf.keras.utils.to_categorical(dataY)

##P4.Construcción del modelo RNN

---
definimos nuestro modelo LSTM: 
Aquí definimos una única capa LSTM oculta con 256 unidades de memoria. La red usa deserción con una probabilidad de 20. La capa de salida es una capa densa que usa la función de activación softmax para generar una predicción de probabilidad para cada uno de los 54 caracteres entre 0 y 1.


In [11]:
# define the LSTM model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))    #creamos una capa con 256 unidades de memoria
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))            #Softmax convierte un vector de valores en una distribución de probabilidad para cada uno de los 39
                                                                              #Softmax se utiliza a menudo como la activación para la última capa de una red de clasificación
#utilizamos el algoritmo de optimización de ADAM para la velocidad
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P4.1 Creando chekpoints

---

La red es lenta de entrenar (alrededor de 300 segundos por época) teniendo activa la GPU, ASí que crearemos CHECKPOINTS (puntos de control) para registrar todos los pesos de la red para archivar cada vez que se observe una mejora en la pérdida al final de la época. Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección

In [13]:
# definimos  una carpeta para guardar los checkpoint
filepath="checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

###P4.2 entrenando

In [15]:
history = model.fit(X, y, epochs=1000, batch_size=128, verbose=1,callbacks=callbacks_list)

Epoch 1/1000
25/25 [==============================] - ETA: 0s - loss: 0.0071
Epoch 00001: loss did not improve from 0.00305
25/25 [==============================] - 1s 29ms/step - loss: 0.0071
Epoch 2/1000
23/25 [==========================>...] - ETA: 0s - loss: 0.0065
Epoch 00002: loss did not improve from 0.00305
25/25 [==============================] - 1s 27ms/step - loss: 0.0063
Epoch 3/1000
23/25 [==========================>...] - ETA: 0s - loss: 0.0058
Epoch 00003: loss did not improve from 0.00305
25/25 [==============================] - 1s 27ms/step - loss: 0.0057
Epoch 4/1000
23/25 [==========================>...] - ETA: 0s - loss: 0.0081
Epoch 00004: loss did not improve from 0.00305
25/25 [==============================] - 1s 27ms/step - loss: 0.0078
Epoch 5/1000
24/25 [===========================>..] - ETA: 0s - loss: 0.0053
Epoch 00005: loss did not improve from 0.00305
25/25 [==============================] - 1s 26ms/step - loss: 0.0053
Epoch 6/1000
23/25 [===============

##P5.Generando texto con una red LSTM


---
Vamos a cargar el ultimo CHECKPOINT de entrenamiento y con el haremos MAGIA!!!


In [16]:
filename = "/content/checkpoints/weights-improvement-935-0.0031.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P5.1 mapeo inverso (números a letras)
creamos un mapeo inverso que podamos usar para convertir los números enteros nuevamente en caracteres para que podamos entender las predicciones

In [17]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

###P5.2 hacer predicciones
La forma más sencilla de utilizar el modelo Keras LSTM para hacer predicciones es comenzar primero con una secuencia semilla como entrada, generar el siguiente carácter y luego actualizar la secuencia semilla para agregar el carácter generado al final y recortar el primer carácter. Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1000 caracteres de longitud).

In [20]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
#start ="soy un cerdito"
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de 10000 caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"o r t u n a .   e l   m á s   g r a n d e   l e s   d i j o   a   s u s   h e r m a n o s   q u e  "

a sus hermanos, - construiré mi casa en un santiamén con todos estos troncos y me iré también a jugar.  cuando las tres casitas estuvieron terminadas, los cerditos cantaban y bailaban en la puerta, felices por haber acabado con el problema:  -¡quién teme al lobo feroz, al lobo, al lobo!  - ¡quién teme al lobo feroz, al lobo feroz! - cantaban desde dentro los cerditos.  de nuevo el lobo, más enfurecido que antes al sentirse engañado, se colocó delante de la puerta y comenzó a soplar y soplar gruñendo: - ¡cerditos, abridme la puerta! - no, no,
Done.


##P6.Mejorando la red (una LSTM más grande)

In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))    #creamos una capa con 256 unidades de memoria
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(256))    #creamos una capa con 256 unidades de memoria
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))            #Softmax convierte un vector de valores en una distribución de probabilidad para cada uno de los 39
                                                                              #Softmax se utiliza a menudo como la activación para la última capa de una red de clasificación
model.compile(loss='categorical_crossentropy', optimizer='adam')
#cambiamos el nombre de archivo de los pesos con puntos de control para que 
#podamos distinguir entre los pesos de esta red 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

###P6.1 mejoramos el entrenamiento

---
aumentamos las epoch y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender.


In [24]:
#los tiempos de entrenamiento aumentaran al doble que en la versión anterior
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
48/49 [============================>.] - ETA: 0s - loss: 3.0906
Epoch 00001: loss did not improve from 0.00244
49/49 [==============================] - 6s 53ms/step - loss: 3.0910
Epoch 2/50
48/49 [============================>.] - ETA: 0s - loss: 2.9871
Epoch 00002: loss did not improve from 0.00244
49/49 [==============================] - 2s 49ms/step - loss: 2.9873
Epoch 3/50
49/49 [==============================] - ETA: 0s - loss: 2.9851
Epoch 00003: loss did not improve from 0.00244
49/49 [==============================] - 2s 47ms/step - loss: 2.9851
Epoch 4/50
49/49 [==============================] - ETA: 0s - loss: 2.9820
Epoch 00004: loss did not improve from 0.00244
49/49 [==============================] - 2s 45ms/step - loss: 2.9820
Epoch 5/50
48/49 [============================>.] - ETA: 0s - loss: 2.9773
Epoch 00005: loss did not improve from 0.00244
49/49 [==============================] - 2s 43ms/step - loss: 2.9772
Epoch 6/50
49/49 [===========================

###P6.2 haciendo predicciones

In [26]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"e   e n g a ñ a d o ,   s e   c o l o c ó   d e l a n t e   d e   l a   p u e r t a   y   c o m e n"
zó a soplar y soplar greñ hasii s estrue de madera    puedo encontrar un montón de madera por los alrededores - explicó 
a sus hermanos, - construiré mi casa en un santiamén me canita de ladrillo mequeñ y lo conlegra. se luertaa y los   - ¡cerditos, abridme la puerta! - no, no, no, no te vamos a abrir. -  pues si no me abrís... ¡soplaré y soplaré y la casita derribaré!  y se puso a soplar gan fuerte como el viento de invierno. sopló y sopló  pero la casita de ladrillo me mu hermano madoa le vuerta   e sus hermanos, - construiré mi casa en un santiamén me canita de ladrillo mequeñ y lo conlegra. se luertaa y los   - ¡cerditos, abridme la puerta! - no, no, no, no te vamos a abrir. -  pues si no me abrís... ¡soplaré y soplaré y la casita derribaré!  y se puso a soplar gan fuerte como el viento de invierno. sopló y sopló  pero la casita de ladrillo me mu hermano madoa le vuerta   

##P7. exportar modelo RNN

In [ ]:
!pip install h5py

In [ ]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modeloRNN_cuentosPesos.hdf5")
model.save('modelRNN_cuentos_v_h5.h5')
print("modelo salvado en disco")

modelo salvado en disco


###P7.1 cargando un modelo

In [ ]:
# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('/content/modelRNN_cuentos_v_h5.h5')

In [ ]:
chars = sorted(list(set("comiendo una manzana")))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))
pattern = dataX[5]
print(pattern)

En total hay 55827 caracteres, y el diccionario tiene un tamaño de 11 caracteres.
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 18, 16, 31, 20, 33, 36, 18, 24, 35, 16, 2, 33, 30, 25, 16, 1, 0, 23, 16, 17, 50, 16, 2, 36, 29, 16, 2, 37, 20, 41, 2, 36, 29, 16, 2, 19, 36, 27, 18, 20, 2, 29, 24, 51, 16, 2, 32, 36, 20, 2, 32, 36, 20, 33, 50, 16, 2, 28, 36, 18, 23, 30, 2, 16, 2, 34, 36, 2, 28, 16, 19, 33, 20, 2, 40, 2, 16, 2, 34, 36, 2, 16, 17, 36]


In [ ]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = new_model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"n t e m p l a r   l a s   f l o r e s ,   q u e   c o m e n z a b a n   a   l l e n a r l o   t o d o .   a l l í   v i o   e n   e l   e s t a n q u e   d o s   d e   a q u e l l o s   p á j a r o s"
 sue a la belle y con la boca abierta, lus amlgos de la casa de los piratas, el pey y el gato com su camino de la casa de su harfio, el príncipe juan, pirocho y la reñora darling se quedó hacia ella.
y se precó ticme que el cartillo y se puedó hacia la casa de su harta que se encontraban en el estuvendo conmigo y sodos que el carciici y se puedó hacia ella.
el patito se sintió delta en palacio, pero el hombre de jengibre corrió más rápido. en la cocina de la casa de los piratas, el peys de noche y la reñora darling se quedó hacia la casa de los piratas,
una teca a tu padre la boca abierta dl la casa y algo suppirar y al canbio, estaba muy bueno y poder vup hijas de podenas del príncipe pue al verla. se acuirtió cnn un cama y deseruer a la princesa y lls piratas a la princesa y l



---



---



#Ejemplo 2: generar texto de cuentos, usando Keras

##P0. importar librerias

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

##P0. Descarga y preprocesado de los datos

In [ ]:
fileDL= tf.keras.utils.get_file('LasMinasDelReySalomon.txt','https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/Datasets/Panel_Txt_Files/LasMinasDelReySalomon.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')
texto = texto.lower()

##P1. entendiendo el texto

In [ ]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:505098 caracteres
el texto esta compuesto de estos :63 caracteres
['\n', ' ', '!', '"', '&', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '\\', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '“']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: ' \n \n \n \n \n \n \n \n  \n \nhenry r. haggard \n \n \n \n \n \n \nlas minas del rey salomón \n   \n  \n  \n   \n   \n    '
array([ 1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,
        1,  0,  1,  0, 34, 31, 40, 44, 51,  1, 44,  9,  1, 34, 27, 33, 33,
       27, 44, 30,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,
       38, 27, 45,  1, 39, 35, 40, 27, 45,  1, 30, 31, 38,  1, 44, 31, 51,
        1, 45, 27, 38, 41, 39, 59, 40,  1,  0,  1,  1,  1,  0,  1,  1,  0,
        1,  1,  0,  1,  1,  1,  0,  1,  1,  1,  0,  1,  1,  1,  1])


##P3. preparar los datos para ser usados en la RNN

In [ ]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

' \n \n \n \n \n \n \n \n  \n \nhenry r. haggard \n \n \n \n \n \n \nlas minas del rey salomón \n   \n  \n  \n   \n   \n    introducción \n   \n   \n    ahora que este libro está'
' impreso y a punto de salir al mundo, ejerce sobre mí un \nenorme peso la conciencia de sus defectos, tanto de estilo como de contenido. en lo \nreferent'
'e a este último, sólo puedo decir que no pretende ser una relación exhaustiva de \ntodo lo que vimos e hicimos. hay muchas cosas concernientes a nuestro'
' viaje a \nkukuanalandia en las que me hubiese gustado explayarme y a las que, de hecho, apenas \naludo. entre ellas se encuentran las curiosas leyendas '
'que recogí sobre las armaduras \nque nos salvaron de la muerte en la gran batalla de loo, y también sobre los silenciosos \no colosos de la entrada de la'
' cueva de estalactitas. por otra parte, si me hubiera dejado \nllevar por mis inclinaciones, me habría gustado ahondar en las diferencias, algunas de \nl'
'as cuales me resultan muy sugestivas, en

###P3.1 separar los datos en agrupamientos (batches)

In [ ]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  ' \n \n \n \n \n \n \n \n  \n \nhenry r. haggard \n \n \n \n \n \n \nlas minas del rey salomón \n   \n  \n  \n   \n   \n    introducción \n   \n   \n    ahora que este libro est'
Target data:  '\n \n \n \n \n \n \n \n  \n \nhenry r. haggard \n \n \n \n \n \n \nlas minas del rey salomón \n   \n  \n  \n   \n   \n    introducción \n   \n   \n    ahora que este libro está'


In [ ]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=build_model(vocab_size=vocab_size,
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=
#    -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16128     
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_3 (Dense)              (64, None, 63)            64575     
Total params: 5,327,679
Trainable params: 5,327,679
Non-trainable params: 0
_________________________________________________________________


##P4. Entrenando la RNN

In [ ]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
#con los argumentos por defecto del optimizador Adam.  
model.compile(optimizer='adam',loss=loss)

###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.


In [ ]:
checkpoint_dir='/content/checkpointV2'
checkpoint_prefix= os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

###P4.2 entrenando

In [ ]:
EPOCHS=500
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

##P5. Generando texto nuevo usando la RNN

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.5  
  #entre más alta la temperatura más creatividad al modelo, pero tambien
  #más errores ortograficos.
  model.reset_states()
  #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [ ]:
print(generate_text(model, start_string=u"la colonia europea"))

estirpe britanicador 
en la atroumizado autor inversabe el foso de la maÃ±ana, 
que el regimiento terrible no me gusta falla no podÃ­amos distinguir las cazadoras 
de todos los hombres blancos a la mandÃ­bula con un hombre alto, de mayer, es de su parte, si es que 
llegasticio estÃ¡bamos lispuas por su tipo con un suspiro de alivio, y 
no podÃ­amos seguir por allÃ­, a menos que debÃ­amos peser la amplia avenida cuya 
con una cabaÃ±a al que comercial atrÃ¡s, fero no podÃ­amos 
recibir la larga y aplanada su filtr


##P6.exportando modelo

In [ ]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentosV2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modelRNN_cuentosV2_pesos.hdf5")
model.save('modelRNN_cuentosV2.h5')
print("modelo salvado en disco")

modelo salvado en disco
